In [1]:
sc

<SparkContext master=local[*] appName=PySparkShell>

# Delete empty folders (no reviews)

Received data has a bunch of empty folders for timestamps where no new reviews were received. In this section we remove them so later processing will have be easier.

In [7]:
import os
import shutil

In [2]:
def get_folder_size(folder):
    total_size = 0
    for dirpath, dirnames, filenames in os.walk(folder):
        for f in filenames:
            fp = os.path.join(dirpath, f)
            total_size += os.path.getsize(fp)
    return total_size

In [3]:
def delete_small_folders(parent_dir, size_limit):
    for root, dirs, files in os.walk(parent_dir, topdown=False):
        for dir_name in dirs:
            dir_path = os.path.join(root, dir_name)
            folder_size = get_folder_size(dir_path)
            if folder_size < size_limit:
                print(f"Deleting folder: {dir_path}")
                shutil.rmtree(dir_path)

In [4]:
#Empty folders, with no reviews, will have file size of 8 bytes.
parent_dir = r"C:\Users\jef-w\Desktop\Uni\KUL\Year_1\Advanced_Analytics\Assignments\Assignment_3\spark\notebooks\Raw_Data\2023-04-11-12-45"
size_limit = 9  # Specify the size limit in bytes
delete_small_folders(parent_dir, size_limit)

Deleting folder: C:\Users\jef-w\Desktop\Uni\KUL\Year_1\Advanced_Analytics\Assignments\Assignment_3\spark\notebooks\Data\2023-04-11-12-45\2023-04-11-12-45-1681210000000
Deleting folder: C:\Users\jef-w\Desktop\Uni\KUL\Year_1\Advanced_Analytics\Assignments\Assignment_3\spark\notebooks\Data\2023-04-11-12-45\2023-04-11-12-45-1681210010000
Deleting folder: C:\Users\jef-w\Desktop\Uni\KUL\Year_1\Advanced_Analytics\Assignments\Assignment_3\spark\notebooks\Data\2023-04-11-12-45\2023-04-11-12-45-1681210020000
Deleting folder: C:\Users\jef-w\Desktop\Uni\KUL\Year_1\Advanced_Analytics\Assignments\Assignment_3\spark\notebooks\Data\2023-04-11-12-45\2023-04-11-12-45-1681210030000
Deleting folder: C:\Users\jef-w\Desktop\Uni\KUL\Year_1\Advanced_Analytics\Assignments\Assignment_3\spark\notebooks\Data\2023-04-11-12-45\2023-04-11-12-45-1681210040000
Deleting folder: C:\Users\jef-w\Desktop\Uni\KUL\Year_1\Advanced_Analytics\Assignments\Assignment_3\spark\notebooks\Data\2023-04-11-12-45\2023-04-11-12-45-168121

Deleting folder: C:\Users\jef-w\Desktop\Uni\KUL\Year_1\Advanced_Analytics\Assignments\Assignment_3\spark\notebooks\Data\2023-04-11-12-45\2023-04-11-12-45-1681213550000
Deleting folder: C:\Users\jef-w\Desktop\Uni\KUL\Year_1\Advanced_Analytics\Assignments\Assignment_3\spark\notebooks\Data\2023-04-11-12-45\2023-04-11-12-45-1681213560000
Deleting folder: C:\Users\jef-w\Desktop\Uni\KUL\Year_1\Advanced_Analytics\Assignments\Assignment_3\spark\notebooks\Data\2023-04-11-12-45\2023-04-11-12-45-1681213570000
Deleting folder: C:\Users\jef-w\Desktop\Uni\KUL\Year_1\Advanced_Analytics\Assignments\Assignment_3\spark\notebooks\Data\2023-04-11-12-45\2023-04-11-12-45-1681213580000
Deleting folder: C:\Users\jef-w\Desktop\Uni\KUL\Year_1\Advanced_Analytics\Assignments\Assignment_3\spark\notebooks\Data\2023-04-11-12-45\2023-04-11-12-45-1681213590000
Deleting folder: C:\Users\jef-w\Desktop\Uni\KUL\Year_1\Advanced_Analytics\Assignments\Assignment_3\spark\notebooks\Data\2023-04-11-12-45\2023-04-11-12-45-168121

Deleting folder: C:\Users\jef-w\Desktop\Uni\KUL\Year_1\Advanced_Analytics\Assignments\Assignment_3\spark\notebooks\Data\2023-04-11-12-45\2023-04-11-12-45-1681216340000
Deleting folder: C:\Users\jef-w\Desktop\Uni\KUL\Year_1\Advanced_Analytics\Assignments\Assignment_3\spark\notebooks\Data\2023-04-11-12-45\2023-04-11-12-45-1681216350000
Deleting folder: C:\Users\jef-w\Desktop\Uni\KUL\Year_1\Advanced_Analytics\Assignments\Assignment_3\spark\notebooks\Data\2023-04-11-12-45\2023-04-11-12-45-1681216360000
Deleting folder: C:\Users\jef-w\Desktop\Uni\KUL\Year_1\Advanced_Analytics\Assignments\Assignment_3\spark\notebooks\Data\2023-04-11-12-45\2023-04-11-12-45-1681216370000
Deleting folder: C:\Users\jef-w\Desktop\Uni\KUL\Year_1\Advanced_Analytics\Assignments\Assignment_3\spark\notebooks\Data\2023-04-11-12-45\2023-04-11-12-45-1681216380000
Deleting folder: C:\Users\jef-w\Desktop\Uni\KUL\Year_1\Advanced_Analytics\Assignments\Assignment_3\spark\notebooks\Data\2023-04-11-12-45\2023-04-11-12-45-168121

Deleting folder: C:\Users\jef-w\Desktop\Uni\KUL\Year_1\Advanced_Analytics\Assignments\Assignment_3\spark\notebooks\Data\2023-04-11-12-45\2023-04-11-12-45-1681219620000
Deleting folder: C:\Users\jef-w\Desktop\Uni\KUL\Year_1\Advanced_Analytics\Assignments\Assignment_3\spark\notebooks\Data\2023-04-11-12-45\2023-04-11-12-45-1681219630000
Deleting folder: C:\Users\jef-w\Desktop\Uni\KUL\Year_1\Advanced_Analytics\Assignments\Assignment_3\spark\notebooks\Data\2023-04-11-12-45\2023-04-11-12-45-1681219640000
Deleting folder: C:\Users\jef-w\Desktop\Uni\KUL\Year_1\Advanced_Analytics\Assignments\Assignment_3\spark\notebooks\Data\2023-04-11-12-45\2023-04-11-12-45-1681219650000
Deleting folder: C:\Users\jef-w\Desktop\Uni\KUL\Year_1\Advanced_Analytics\Assignments\Assignment_3\spark\notebooks\Data\2023-04-11-12-45\2023-04-11-12-45-1681219660000
Deleting folder: C:\Users\jef-w\Desktop\Uni\KUL\Year_1\Advanced_Analytics\Assignments\Assignment_3\spark\notebooks\Data\2023-04-11-12-45\2023-04-11-12-45-168121

Deleting folder: C:\Users\jef-w\Desktop\Uni\KUL\Year_1\Advanced_Analytics\Assignments\Assignment_3\spark\notebooks\Data\2023-04-11-12-45\2023-04-11-12-45-1681222880000
Deleting folder: C:\Users\jef-w\Desktop\Uni\KUL\Year_1\Advanced_Analytics\Assignments\Assignment_3\spark\notebooks\Data\2023-04-11-12-45\2023-04-11-12-45-1681222890000
Deleting folder: C:\Users\jef-w\Desktop\Uni\KUL\Year_1\Advanced_Analytics\Assignments\Assignment_3\spark\notebooks\Data\2023-04-11-12-45\2023-04-11-12-45-1681222900000
Deleting folder: C:\Users\jef-w\Desktop\Uni\KUL\Year_1\Advanced_Analytics\Assignments\Assignment_3\spark\notebooks\Data\2023-04-11-12-45\2023-04-11-12-45-1681222910000
Deleting folder: C:\Users\jef-w\Desktop\Uni\KUL\Year_1\Advanced_Analytics\Assignments\Assignment_3\spark\notebooks\Data\2023-04-11-12-45\2023-04-11-12-45-1681222920000
Deleting folder: C:\Users\jef-w\Desktop\Uni\KUL\Year_1\Advanced_Analytics\Assignments\Assignment_3\spark\notebooks\Data\2023-04-11-12-45\2023-04-11-12-45-168122

# Combine Received files into single dataframe

Here we take all the individual received text files and combine them into a single, easy to use csv.
We also clean the text of the reviews of newlines, so it can easily be converted into csv.

In [84]:
#Example of single dataframe for later reference
path = r"C:\Users\jef-w\Desktop\Uni\KUL\Year_1\Advanced_Analytics\Assignments\Assignment_3\spark\notebooks\Raw_Data\2023-04-11-12-45\2023-04-11-12-45-1681209930000"
df1 = spark.read.json(path)
df1.show()

+-------+-----+---------+--------------------+
| app_id|label|review_id|         review_text|
+-------+-----+---------+--------------------+
|2248580|    1|136512581|A Delightful and ...|
|1934040|    1|136513114|This is the only ...|
+-------+-----+---------+--------------------+



In [64]:
import os
import shutil
from pyspark.sql import functions as F

In [76]:
#Function for getting the first dataframe of a folder, so we can concatenate it with the rest
def initialize_dataframe(folder_path):
    for dirpath,dirnames,filenames in os.walk(folder_path):
        for dirname in dirnames:
            subpath = os.path.join(dirpath, dirname)
            df = spark.read.json(subpath)
            
            # Here we remove new lines from the text
            df = df.withColumn("review_text",F.translate("review_text",'\n',' '))
            
            return df

def combine_into_dataframe(folder_path):
    df = initialize_dataframe(folder_path)
    firstdir = True
    
    for dirpath,dirnames,filenames in os.walk(folder_path):
        for dirname in dirnames:
            # We already have the first review, so skip that one
            if(firstdir):
                firstdir = False
                continue
            subpath = os.path.join(dirpath, dirname)
            df1 = spark.read.json(subpath)
            df1 = df1.withColumn("review_text",F.translate("review_text",'\n',''))
            
            df = df.union(df1)
    return df

In [77]:
#Combine all reviews of a folder into a single dataframe
path = r"C:\Users\jef-w\Desktop\Uni\KUL\Year_1\Advanced_Analytics\Assignments\Assignment_3\spark\notebooks\Data\2023-04-11-12-45"
df = combine_into_dataframe(path)
df.show()

+-------+-----+---------+--------------------+
| app_id|label|review_id|         review_text|
+-------+-----+---------+--------------------+
|2248580|    1|136512581|A Delightful and ...|
|1934040|    1|136513114|This is the only ...|
|2334560|    0|136513050|The concept and a...|
|1400910|    1|136514066|Waited for 1.0 to...|
|2248580|    1|136512233|OMG, this game is...|
|2364250|    1|136512230|It's like a tough...|
|1409300|    0|136513866|This game suppose...|
|1729900|    1|135237351|[h1]Before I play...|
|1729900|    1|122580797|cool, has potenti...|
|1729900|    1|125559837|ms paint rhythm m...|
|1729900|    0|121230865|Rhythm Brawl, unl...|
|1729900|    1|121311975|I get more points...|
|1729900|    0|118681190|Rhythm Brawl is a...|
|1729900|    0|119682629|Now let me get th...|
|1729900|    1|116989907|Hey. It's really ...|
|1729900|    1|114308657|Although I found ...|
|1729900|    1|115583159|When I first play...|
|1729900|    1|116734222|Its not finished ...|
|1729900|    

In [ ]:
# Save dataframe as a bunch of separate csv's
# I do not understand why you would want to do this, maybe it's important for later? Weird
folders = path.split(os.path.sep)
last_folder = folders[-1]
outputname = "output_"+last_folder
df.write.csv(outputname)

In [82]:
# Combine all the reviews in one big beautiful csv
df.coalesce(1).write.option("header",True).csv("full_csv_"+last_folder)